<a href="https://colab.research.google.com/github/hissain/mlworks/blob/main/codes/Encoder_Decoder_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import
%matplotlib inline
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

In [ ]:
torch.manual_seed(0)

In [ ]:
# Define the Encoder
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)

    def forward(self, input_seqs, hidden):
        embedded = self.embedding(input_seqs)
        output, hidden = self.lstm(embedded, hidden)
        return output, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size),
                torch.zeros(self.num_layers, batch_size, self.hidden_size))


In [ ]:
# Define the Decoder
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers=1):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input_seqs, hidden):
        embedded = self.embedding(input_seqs)
        output, hidden = self.lstm(embedded, hidden)
        output = self.fc(output)
        return output, hidden


In [ ]:
# Define the Seq2Seq Model
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seqs, target_seqs):
        batch_size = input_seqs.size(0)
        target_length = target_seqs.size(1)
        target_vocab_size = self.decoder.fc.out_features

        encoder_hidden = self.encoder.init_hidden(batch_size)
        encoder_output, encoder_hidden = self.encoder(input_seqs, encoder_hidden)

        decoder_input = torch.tensor([[SOS_token]] * batch_size, dtype=torch.long)
        decoder_hidden = encoder_hidden

        outputs = torch.zeros(batch_size, target_length, target_vocab_size)

        for t in range(target_length):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[:, t, :] = decoder_output.squeeze(1)
            decoder_input = target_seqs[:, t].unsqueeze(1)

        return outputs


Here
- We define an `EncoderLSTM` class, which takes input sequences and produces hidden states using an LSTM layer.
- We define a `DecoderLSTM` class, which takes hidden states from the encoder and generates output sequences using another LSTM layer followed by a fully connected layer.
- We define a `Seq2Seq` class that encapsulates the encoder and decoder. It takes input sequences and target sequences, passes the input sequences through the encoder, and then feeds the hidden states to the decoder to generate output sequences.